In [2]:
import torch
import torch.nn as nn
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.optim as optim
import torch.nn.functional as F
# from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import models
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
# import time
import cv2
import os


In [3]:
for dirname, _, filenames in os.walk('/kaggle/input/cifar10-python'):
    for foldername in _:
        print(os.path.join(dirname, foldername))
        break

/kaggle/input/cifar10-python/cifar-10-batches-py


In [5]:
transform = transforms.Compose(
    [transforms.RandomResizedCrop(255),
     transforms.CenterCrop(224),  
     transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_len=len(trainset)
test_len=len(testset)

In [7]:
print(test_len)

10000


In [8]:
model = models.resnet50(pretrained=True)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s] 


In [9]:
for param in model.parameters(): # freezes the layers
    param.required_grad = False
number_feature = model.fc.in_features
model.fc = torch.nn.Linear(in_features=number_feature , out_features=10, bias=True)

In [10]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device is ', device)

device is  cuda


In [11]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [16]:
model.train()
    
for epoch in range(15):
    total_correct = 0.0 
    running_loss = 0.0 
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.cuda(), labels.cuda()
        # Forward pass
        output = model(inputs)
        # Get predicted classes
        output_idex = torch.argmax(output, dim=1)
        # Calculate the number of correct predictions
        total_correct += (labels == output_idex).sum().item()
        # Zero the parameter gradients
        optimizer.zero_grad()
        # Calculate loss
        loss = criterion(output, labels)  # Fixed typo: 'cirterion' to 'criterion'
        running_loss += loss.item() * inputs.size(0)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    print('epoch: ', epoch, 'loss: ', running_loss / train_len, 'accuracy: ', (total_correct / train_len) * 100, '%')

    if (epoch + 1) % 3 == 0:
            Data_Snapshot = f'/kaggle/working/cnnnet_epoch_{epoch+1}.pth'
            torch.save(model.state_dict(), Data_Snapshot)
            print(f" saved at {Data_Snapshot}")
    print("---------------------------")

print('finished training')


498.9s 14 epoch:  0 loss:  1.6747970003509522 accuracy:  38.506 %
498.9s 15 ---------------------------
978.6s 16 epoch:  1 loss:  1.334530128326416 accuracy:  52.318 %
978.6s 17 ---------------------------
1457.4s 18 epoch:  2 loss:  1.1832898192214967 accuracy:  58.104 %
1457.4s 19 saved at /kaggle/working/cnnnet_epoch_3.pth
1457.4s 20 ---------------------------
1935.5s 21 epoch:  3 loss:  1.0852978984069823 accuracy:  61.692 %
1935.5s 22 ---------------------------
2412.5s 23 epoch:  4 loss:  1.0109545670890807 accuracy:  64.364 %
2412.5s 24 ---------------------------
2888.3s 25 epoch:  5 loss:  0.94886798620224 accuracy:  66.806 %
2888.3s 26 saved at /kaggle/working/cnnnet_epoch_6.pth
2888.3s 27 ---------------------------
3363.0s 28 epoch:  6 loss:  0.8991863755226135 accuracy:  68.794 %
3363.0s 29 ---------------------------
3836.9s 30 epoch:  7 loss:  0.8649814581489563 accuracy:  69.746 %
3836.9s 31 ---------------------------
4309.9s 32 epoch:  8 loss:  0.8314949030303955 ac

In [20]:
# Test

with torch.no_grad():
    model.eval()
    total_loss=0.0 
    total_correct=0.0 
    for inputs, labels in testloader:
        labels=labels.to(device)
        outputs=model(inputs.to(device))
        loss=criterion(outputs, labels)
        total_loss+=loss.item()*inputs.size(0)
        output_idx=torch.argmax(outputs, dim=1)
        total_correct+=sum(labels==output_idx).sum().item() 
        accuracy = (total_correct / test_len) * 100
        loss = total_loss / test_len

    print(f'Accuracy: {accuracy:.2f}% Loss: {loss:.4f}')

Accuracy: 50.02% Loss: 2.3496
